In [ ]:
# Upload CSV from your computer
from google.colab import files
uploaded = files.upload()


In [ ]:
df = pd.read_csv(io.BytesIO(uploaded['Parkinsson_speech_dataset (1).csv']))

In [ ]:
import pandas as pd
import io

df = pd.read_csv(io.BytesIO(uploaded["Parkinsson_speech_dataset (1).csv"]))
df.head()

# **PreProcess The Data**

In [ ]:
df.columns

In [ ]:
# Drop unnecessary identifier columns
df = df.drop(columns=['id'], errors='ignore')

# Split features (X) and labels (y)
X = df.drop(columns=['class'])  # Features
y = df['class']  # Labels (1 = Parkinson's, 0 = Healthy)

# Normalize the features
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print("✅ Features shape:", X_scaled.shape)
print("✅ Labels shape:", y.shape)


# **Train-Test Split & Model Training**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

# Train a Random Forest model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate
print("✅ Accuracy:", accuracy_score(y_test, y_pred))
print("\n🔍 Classification Report:\n", classification_report(y_test, y_pred))
print("\n📊 Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


**Save the Model for later use**

In [ ]:
import joblib

# Save model and scaler
joblib.dump(model, 'parkinsons_model.pkl')
joblib.dump(scaler, 'scaler.pkl')
print("✅ Model and scaler saved successfully.")


In [ ]:
from google.colab import files
uploaded = files.upload()


In [ ]:
import zipfile
import os

# Extract the uploaded file
with zipfile.ZipFile("EYE DATASET ZIPPED.zip", 'r') as zip_ref:
    zip_ref.extractall("mrlEyes_2018")

print("✅ Dataset extracted successfully.")


# **Preprocessing Code**

In [ ]:
import os

# List all subdirectories in current working dir
for folder in os.listdir():
    print(folder)

In [ ]:
import os

# Show a few .png image filenames to confirm label pattern
sample_files = []
for root, dirs, files in os.walk("mrlEyes_2018"):
    for file in files:
        if file.endswith(".png"):
            sample_files.append(os.path.join(root, file))
        if len(sample_files) >= 10:
            break

print("🔍 Sample image file paths:")
for f in sample_files:
    print(f)


In [ ]:
data = []
labels = []
IMG_SIZE = 24
dataset_path = "mrlEyes_2018/data"

for root, dirs, files in os.walk(dataset_path):
    for file in files:
        if file.endswith(".png"):
            filepath = os.path.join(root, file)

            # Label from folder name
            if "sleepy" in root.lower():
                label = 0  # Closed
            elif "awake" in root.lower():
                label = 1  # Open
            else:
                print("❌ Unknown label folder:", root)
                continue

            img = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
            if img is None:
                continue

            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            data.append(img)
            labels.append(label)

print("✅ Total images loaded:", len(data))


✅ Total images loaded: 84898


In [ ]:
from sklearn.model_selection import train_test_split

data = np.array(data).reshape(-1, IMG_SIZE, IMG_SIZE, 1) / 255.0
labels = np.array(labels)

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)


# **Train an Eye State Detection Model **

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout


In [ ]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')  # Binary classification
])


In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [ ]:
history = model.fit(X_train, y_train,
                    epochs=10,
                    validation_data=(X_test, y_test)
                    )


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)


In [ ]:
model.save('eye_state_model.h5')


In [ ]:
import cv2
import tensorflow as tf

# Load the pre-trained model
model = tf.keras.models.load_model('eye_state_model.h5')

# Initialize webcam
cap = cv2.VideoCapture(0)

# Set image size (replace this with the actual IMG_SIZE you used during training, e.g., 24)
IMG_SIZE = 24

while True:
    # Read frame from webcam
    ret, frame = cap.read()

    if not ret:
        print("Failed to capture image")
        break

    # Preprocess frame
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    resized_frame = cv2.resize(gray_frame, (IMG_SIZE, IMG_SIZE)) / 255.0
    reshaped_frame = resized_frame.reshape(1, IMG_SIZE, IMG_SIZE, 1)

    # Make prediction
    prediction = model.predict(reshaped_frame)

    # Show prediction on the frame
    label = "Open Eye" if prediction > 0.5 else "Closed Eye"
    color = (0, 255, 0) if prediction > 0.5 else (0, 0, 255)

    # Display text on the frame
    cv2.putText(frame, label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2, cv2.LINE_AA)

    # Optionally draw a bounding box (for demonstration)
    cv2.rectangle(frame, (50, 50), (300, 300), color, 2)  # Modify coordinates based on detection

    # Display the frame
    cv2.imshow('Eye State Detection', frame)

    # Exit loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


In [ ]:
model = tf.keras.models.load_model('eye_state_model.h5')


In [ ]:
import cv2
import tensorflow as tf

# Load the pre-trained model (ensure the model path is correct)
model = tf.keras.models.load_model('eye_state_model.h5')

# Define image size
IMG_SIZE = 64  # Adjust as per your model input size

# Initialize webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()  # Capture each frame

    if not ret:
        break

    # Preprocess frame
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    resized_frame = cv2.resize(gray_frame, (IMG_SIZE, IMG_SIZE)) / 255.0
    reshaped_frame = resized_frame.reshape(1, IMG_SIZE, IMG_SIZE, 1)

    # Predict eye state
    prediction = model.predict(reshaped_frame)

    # Check if the eye is open or closed
    label = "Open Eye" if prediction > 0.5 else "Closed Eye"

    # Display the prediction on the frame
    cv2.putText(frame, label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

    # Show the frame
    cv2.imshow("Eye State Detection", frame)

    # Break the loop on pressing 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


In [ ]:
pip install flask opencv-python tensorflow


In [ ]:
from flask import Flask, render_template, Response
import cv2
import tensorflow as tf

app = Flask(__name__)

# Load the pre-trained model
model = tf.keras.models.load_model('eye_state_model.h5')

# Initialize webcam
cap = cv2.VideoCapture(0)

# Define image size for preprocessing
IMG_SIZE = 64  # Adjust as per your model input size

# Video streaming function
def gen_frames():
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        # Preprocess the frame
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        resized_frame = cv2.resize(gray_frame, (IMG_SIZE, IMG_SIZE)) / 255.0
        reshaped_frame = resized_frame.reshape(1, IMG_SIZE, IMG_SIZE, 1)

        # Predict eye state
        prediction = model.predict(reshaped_frame)
        label = "Open Eye" if prediction > 0.5 else "Closed Eye"

        # Draw prediction label on the frame
        cv2.putText(frame, label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

        # Encode the frame to send to frontend
        ret, buffer = cv2.imencode('.jpg', frame)
        frame = buffer.tobytes()

        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/video_feed')
def video_feed():
    return Response(gen_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')

if __name__ == '__main__':
    app.run(debug=True)
